In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import linear_model
import statsmodels.formula.api as smf
from sqlalchemy import create_engine
import statsmodels.api as sm
from scipy.stats import bartlett
from scipy.stats import levene
from statsmodels.tsa.stattools import acf

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

import warnings
warnings.filterwarnings(action="ignore")

In [2]:
#get data
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

weather_df = pd.read_sql_query('select * from weatherinszeged',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

In [3]:
#define linear regression model
weather_df['temp_diff'] = weather_df['apparenttemperature'] - weather_df['temperature']
Y = weather_df['temp_diff']
X = weather_df[['humidity', 'windspeed']]
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              temp_diff   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Mon, 16 Sep 2019   Prob (F-statistic):               0.00
Time:                        13:02:57   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4381      0.021    115.948      0.000       2.397       2.479
humidity      -3.0292      0.024   -126.479      0.000      -3.076      -2.982
windspeed     -0.1193      0.001   -176.164      0.000      -0.121      -0.118
==============================================================================
Omnibus:                     3935.747   Durbin-Watson:                   0.267
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4613.311
Skew:                          -0.478   Prob(JB):                         0.00
Kurtosis:                       3.484   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Are the estimated coefficients statistically significant?

The p-values for both humidity and windspeed are 0, so the coefficients for both of those variables are statistically significant.

Are the signs of the estimated coefficients in line with your previous expectations?

I would expect that as humidity increases, the difference between apparent temperature and temperature would increase; however, the sign of the coefficient for humidity is negative, which is not what I would expect. I would expect that as windspeed increases, the difference between apparent temperature and temperature would become more negative, thus decreasing; the coefficient of windspeed in our model is negative, which aligns with this expectation.

Interpret the estimated coefficients. Interpret the estimated coefficients. What are the relations between the target and the explanatory variables?

The constant term of this model is 2.4381, so when there is 0 humidity and 0 windspeed, the difference between apparent temperature and temperature is 2.4381. The coefficient of humidity is -3.0292, so when humidity increases by 1, the difference between apparent temperature and temperature decreases by 3.0292. The coefficient of wind speed is -0.1193, so when the wind speed increases by 1 km/h, the difference between apparent temperature and temperature decreases by 0.1193.

 Next, include the interaction of *humidity* and *windspeed* to the model above and estimate the model using OLS.

In [4]:
#define interaction of humidity and windspeed
weather_df['humid_wind'] = weather_df['humidity'] * weather_df['windspeed']
#define linear regression model
Y2 = weather_df['temp_diff']
X2 = weather_df[['humidity', 'windspeed', 'humid_wind']]
X2 = sm.add_constant(X2)

results2 = sm.OLS(Y2, X2).fit()

results2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              temp_diff   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Mon, 16 Sep 2019   Prob (F-statistic):               0.00
Time:                        13:02:58   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0839      0.033      2.511      0.012       0.018       0.149
humidity       0.1775      0.043      4.133      0.000       0.093       0.262
windspeed      0.0905      0.002     36.797      0.000       0.086       0.095
humid_wind    -0.2971      0.003    -88.470      0.000      -0.304      -0.291
==============================================================================
Omnibus:                     4849.937   Durbin-Watson:                   0.265
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9295.404
Skew:                          -0.378   Prob(JB):                         0.00
Kurtosis:                       4.320   Cond. No.                         193.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Are the coefficients statistically significant? 

The coefficients for humidity, windspeed, and the interaction between them are all statistically significant.

Did the signs of the estimated coefficients for *humidity* and *windspeed* change? 
Yes, the estimated coefficients for humidity and windspeed are now both positive, but the coefficient for the interaction between them is negative.

Interpret the estimated coefficients.
The coefficient for humidity is 0.1775, which means that when the humidity increases by 1, the difference between apparent temperature and temperature increases by 0.1775 degrees C. The coefficient for windspeed is 0.0905, which means that when the windspeed increases by 1 km/h, the difference between apparent temperature and temperature increases by 0.0905 degrees C. The coefficient of the interaction between humidity and windspeed is -0.2971, which much larger than either of the individual factors. This means that if the windspeed increases, we must multiply this value by both the humidity and -0.2971 to get the corresponding change in the difference between apparent temperature and temperature.